In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense , Input
from tensorflow.keras.losses import Loss

In [2]:
xs = np.arange(-5,5,1).astype('float32')
ys = xs * 2 - 1

In [3]:
print(xs.shape)
print(ys.shape)

(10,)
(10,)


## huber loss function without wrapper
In that case we can not pass any type of parameter to the function, thresold is the most important parameter in that type of loss function , so we need to tune this.

In [4]:
def huber_loss(y_true , y_pred):
    thresold = 1
    error = y_true - y_pred
    return_type = tf.abs(error) <= thresold
    r1 = 0.5 * tf.square(error)
    r2 = thresold * ( tf.abs(error) - (0.5*thresold) )
    return tf.where(return_type , r1 , r2)

In [5]:
input = Input(shape=(1,) , name="input_layer")
output_layer = Dense(1 , name="output_layer")(input)

model = tf.keras.models.Model(inputs=input,
                              outputs=output_layer)

model.compile(optimizer="sgd",
              loss=huber_loss)

model.fit(xs,
          ys,
          epochs=500,
          verbose=0)

model.predict([[10.0]])

1/1 [==============================] - 0s 61ms/step


array([[19.023632]], dtype=float32)

## huber loss function with wrapper
In that case we can pass any type of parameter to the function, thresold is the most important parameter in that type of loss function , so we need to tune this parameter.

In [6]:
def huber_loss_wrapper(thresold):
    def huber_loss(y_true , y_pred):
        error = y_true - y_pred
        return_type = tf.abs(error) <= thresold
        r1 = 0.5 * tf.square(error)
        r2 = thresold * (tf.abs(error) - (0.5*thresold))
        return tf.where(return_type , r1 , r2)
    return huber_loss

In [7]:
input = Input(shape=(1,) , name="input_layer")
output_layer = Dense(1 , name="output_layer")(input)

model2 = tf.keras.models.Model(inputs=input,
                               outputs=output_layer)

model2.compile(optimizer="sgd",
               loss=huber_loss_wrapper(thresold=1))

model2.fit(xs,
           ys,
           epochs=500,
           verbose=0)

model2.predict([[10.0]])

1/1 [==============================] - 0s 41ms/step


array([[19.026018]], dtype=float32)

## Own loss Class & Object

In [8]:
class Huber(Loss):
    thresold = 1
    def __init__(self, thresold):
        super().__init__()
        self.thresold = thresold

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        return_type = tf.abs(error) <= self.thresold
        r1 = 0.5 * tf.square(error)
        r2 = self.thresold * (tf.abs(error) - (0.5*self.thresold))
        return tf.where(return_type , r1 , r2)

In [9]:
input = Input(shape=(1,) , name="input_layer")
output_layer = Dense(1 , name="output_layer")(input)

model3 = tf.keras.models.Model(inputs=input,
                               outputs=output_layer)

model3.compile(optimizer="sgd",
               loss=Huber(thresold=1))

model3.fit(xs,
           ys,
           epochs=500,
           verbose=0)

model3.predict([[10.0]])

1/1 [==============================] - 0s 41ms/step


array([[19.017038]], dtype=float32)